In [1]:
import pickle
import numpy as np
import pandas as pd
import time

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

In [3]:
fname = "../WESAD/S2/S2.pkl"

In [34]:
nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

In [44]:
lst = list()
for i in nums:
    fname = f"../WESAD/S{i}/S{i}.pkl"
    raw_data = load_data(fname)
    lst.append(raw_data['signal']['wrist']['EDA'].shape[0])
    print(i,':', raw_data['signal']['wrist']['EDA'].shape[0])
sum(lst)

2 : 24316
3 : 25972
4 : 25692
5 : 25032
6 : 28284
7 : 20952
8 : 21864
9 : 20892
10 : 21984
11 : 20932
13 : 22148
14 : 22192
15 : 21008
16 : 22524
17 : 23680


347472

In [49]:
((20952)*.25)/60

87.3

In [52]:
36*60*15/.25

129600.0

In [39]:
print('4',':', raw_data['signal']['wrist']['TEMP'].shape[0])

4 : 24316


In [35]:
def load_data(fname):
    with open(fname, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        raw_data = u.load()
    return raw_data

In [5]:
num_rows = len(raw_data['signal']['chest']['ACC'])

In [6]:
raw_data['subject']

'S2'

In [7]:
list(raw_data.keys())

['signal', 'label', 'subject']

In [8]:
keys = list(raw_data.keys())
modal_keys = list(raw_data['signal'].keys())
signal_keys = {}
for modal in modal_keys:
    signal_keys[modal] = list(raw_data['signal'][modal].keys())

In [9]:
signal_keys

{'chest': ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'],
 'wrist': ['ACC', 'BVP', 'EDA', 'TEMP']}

In [10]:
subject = np.ones(num_rows) * 2  # S2
labels = raw_data['label']
columns = ['subject', 'label']
for modal in modal_keys:
    for signal in signal_keys[modal]:
        columns.append(modal+'-'+signal)
print(columns)

['subject', 'label', 'chest-ACC', 'chest-ECG', 'chest-EMG', 'chest-EDA', 'chest-Temp', 'chest-Resp', 'wrist-ACC', 'wrist-BVP', 'wrist-EDA', 'wrist-TEMP']


In [23]:
data_chest = {'subject': subject, 'labels': labels}
for key in signal_keys['chest']:
    if key == 'ACC':
        data_chest[key+'_1'] = raw_data['signal']['chest'][key][:, 0]
        data_chest[key+'_2'] = raw_data['signal']['chest'][key][:, 1]
        data_chest[key+'_3'] = raw_data['signal']['chest'][key][:, 2]
    else: data_chest[key] = raw_data['signal']['chest'][key]

In [24]:
data_wrist = {'subject': subject, 'labels': labels}
for key in signal_keys['wrist']:
    if key == 'ACC':
        data_wrist[key+'_1'] = raw_data['signal']['wrist'][key][:, 0]
        data_wrist[key+'_2'] = raw_data['signal']['wrist'][key][:, 1]
        data_wrist[key+'_3'] = raw_data['signal']['wrist'][key][:, 2]
    else: data_wrist[key] = raw_data['signal']['wrist'][key]

In [25]:
data_chest['labels'].shape, data_wrist['labels'].shape

((4255300,), (4255300,))

In [53]:
data_wrist

{'subject': array([2., 2., 2., ..., 2., 2., 2.]),
 'labels': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'ACC_1': array([62., 66., 41., ..., 41., 39., 56.]),
 'ACC_2': array([-21.,  13.,   9., ...,  25.,  27.,  26.]),
 'ACC_3': array([107.,  53.,  15., ...,  11.,  22.,  10.]),
 'BVP': array([[-59.37],
        [-53.42],
        [-44.4 ],
        ...,
        [ 18.26],
        [ 18.68],
        [ 19.71]]),
 'EDA': array([[1.138257],
        [1.125444],
        [1.011405],
        ...,
        [0.059208],
        [0.073303],
        [0.045113]]),
 'TEMP': array([[35.41],
        [35.41],
        [35.41],
        ...,
        [34.23],
        [34.23],
        [34.23]])}

In [28]:
data_wrist['ACC_1'].shape, data_wrist['EDA'].shape, data_wrist['TEMP'].shape, data_wrist['BVP'].shape

((194528,), (24316, 1), (24316, 1), (389056, 1))

In [31]:
133000/15

8866.666666666666

In [12]:
start = time.time()
pd_df = pd.DataFrame(index=np.arange(num_rows))
for colname, col in data.items():
    pd_df[colname] = col
print('time:', time.time() - start)

time: 0.14414167404174805


In [13]:
pd_df.head(5)

,subject,labels,ACC_1,ACC_2,ACC_3,ECG,EMG,EDA,Temp,Resp
0,2.0,0,0.9554,-0.2220,-0.5580,0.021423,-0.004440,5.250549,30.120758,-1.148987
1,2.0,0,0.9258,-0.2216,-0.5538,0.020325,0.004349,5.267334,30.129517,-1.124573
2,2.0,0,0.9082,-0.2196,-0.5392,0.016525,0.005173,5.243301,30.138214,-1.152039
3,2.0,0,0.8974,-0.2102,-0.5122,0.016708,0.007187,5.249405,30.129517,-1.158142
4,2.0,0,0.8882,-0.2036,-0.4824,0.011673,-0.015152,5.286407,30.130951,-1.161194


In [14]:
# Enable Arrow-based columnar data transfers to optimize Python-Java processes
ss.conf.set("spark.sql.execution.arrow.enabled", "true")

In [15]:
# Create Spark DF from Pandas DF
spark_df = ss.createDataFrame(pd_df)

In [16]:
spark_df.printSchema()

root
 |-- subject: double (nullable = true)
 |-- labels: integer (nullable = true)
 |-- ACC_1: double (nullable = true)
 |-- ACC_2: double (nullable = true)
 |-- ACC_3: double (nullable = true)
 |-- ECG: double (nullable = true)
 |-- EMG: double (nullable = true)
 |-- EDA: double (nullable = true)
 |-- Temp: float (nullable = true)
 |-- Resp: double (nullable = true)



In [17]:
spark_df.show()

+-------+------+------------------+--------------------+--------------------+-------------------+-------------------+------------------+---------+-----------------+
|subject|labels|             ACC_1|               ACC_2|               ACC_3|                ECG|                EMG|               EDA|     Temp|             Resp|
+-------+------+------------------+--------------------+--------------------+-------------------+-------------------+------------------+---------+-----------------+
|    2.0|     0|0.9553999900817871|-0.22200000286102295| -0.5579999685287476|   0.02142333984375|-0.0044403076171875|  5.25054931640625|30.120758|-1.14898681640625|
|    2.0|     0|0.9257999658584595| -0.2215999960899353| -0.5537999868392944|   0.02032470703125| 0.0043487548828125|    5.267333984375|30.129517|-1.12457275390625|
|    2.0|     0|0.9082000255584717|-0.21960002183914185| -0.5392000079154968| 0.0165252685546875| 0.0051727294921875|5.2433013916015625|30.138214|-1.15203857421875|
|    2.0| 